**Dashpot** (Damper)
    - Dissipating Spring Force
    - Damping or loss of energy over time
    - e.g. : air drag, mechanical damping, etc
    - Applies when there is a relative velocity between the particles in a spring system

![](https://i.ibb.co/2j72q9L/Screenshot-from-2019-03-08-16-55-11.png)

**Explicit vs Implicit Euler**

**Explicit**
    - The next position completely depends on the parameters of the current timestep
    - Can be problematic if the time step is taken large
        - The system can blow up as the current state may change drastically at a large timestep
        
**Implicit**
    - It depends on the parameters that are evaluated at the end of the timestep instead of the beginning
    - x = f(x)
        - Unknown is the function of its own (as it depends on its own value)
    - **Newton's Method** (A quadratic convergence method)
    

**Newton's method**
- For linear approximation, we use Taylor's function
- Disadv:
    - Can never converge if we start with a random guess that is too far from the original root

![](https://i.ibb.co/0D1DjC0/Screenshot-from-2019-03-11-17-44-52.png)
![](https://i.ibb.co/5K4m7xX/Screenshot-from-2019-03-11-17-53-02.png)

## Implicit Euler

- Can handle changes for large timesteps and maintain the system to be stable
- Costlier than explicit euler at each timestep, but is often found to be productive

- Position, velocity and state when applying Implicit Euler:

![](https://i.ibb.co/qrSZSKX/Screenshot-from-2019-03-26-16-40-53.png)

- Implicit euler and Explicit euler are time-reversed version of each other
- You can retrieve one using the other by following a time-reversed version of the former

### Integrators
- Some integrators are the time-reversed version of itself
- Example:
    ![](https://i.ibb.co/b3KZLSm/Screenshot-from-2019-04-04-17-15-43.png)

### Jacobian Matrix
- A matrix of all first-order partial derivatives of a vector-valued function
- A position (q) matrix, A force (F) matrix, and a Jacobian (J) matrix

# Week 4

### 1. Detecting collisions between two balls

- Detecting a collision
    - Instantaneous approach
    - Continuous duration approach


![](https://i.ibb.co/3dv73Gd/Screenshot-from-2019-04-04-17-43-31.png)
    
### 2. Detecting collision between a ball and a wall


![](https://i.ibb.co/WFyH21W/Screenshot-from-2019-04-04-17-49-07.png)
![](https://i.ibb.co/6NxDGbq/Screenshot-from-2019-04-04-17-51-42.png)


### 3. Detecting collision between a ball and an edge

![](https://i.ibb.co/ZH8Lrbp/Screenshot-from-2019-04-04-18-01-36.png)